<a href="https://colab.research.google.com/github/mcianfrocco/RELION-colab/blob/main/RELION_motion_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CryoEM processing: Motion correction
Welcome to an interactive notebook to introduce motion correction. Here you will be able to: 

1.   Upload cryoEM movies and perform motion correction using RELION
2.   Adjust parameters in motion correction to observe how it affects the resulting aligned micrograph.


# Install RELION
The first step needed to run this RELION colab notebook is to compile RELION from source code. By clicking the 'Play' button below, the notebook will:

1.   Install  the appropriate dependencies
2.   Clone RELION code from GitHub repo 
3.   Compile RELION 

This will take ~5 minutes.

In [ ]:
#@title *Run this cell to set up the software environment*
%%shell
sudo apt install cmake git build-essential mpi-default-bin mpi-default-dev libfftw3-dev libtiff-dev
rm -rf relion/
git clone https://github.com/3dem/relion.git
cd relion/
mkdir build/
cd build
cmake ..
make -j5
pip install mrcfile

# Load movie data
Select movie for motion correction. 

### Option 1: Upload movie & gain reference using Google Drive

In [ ]:
#@title *Run this cell to connect your Google Drive account*
from google.colab import files
from google.colab import drive
import os
drive.mount('/content/drive')

### Option 2: Upload movie & gain reference using web upload

In [ ]:
#@title *Run this cell and click "Choose Files" to upload your movie*
from google.colab import files
file = files.upload()
for fn in file.keys(): Filename=fn
print("\n ---> your filename is "+Filename)

In [ ]:
#@title *Run this cell and click "Choose Files" to upload your gain reference*
from google.colab import files
file = files.upload()
for fn in file.keys(): Filename=fn
print("\n ---> your filename is "+Filename)

# Run motion correction 
Run RELION motion correction and visualize resulting micrograph. 

If you connected your Google Drive account you need to enter the full path to the file, including "/content/drive/.../your_file.tif"

In [ ]:
#@title Run motion correction & display aligned micrograph
#@markdown Input parameters for motion correction
Movie = ''  #@param {type: "string"}
Patch_X = 1  #@param {type: "slider", min: 1, max: 10}
Patch_Y = 1  #@param {type: "slider", min: 1, max: 10}
B_Factor = 1  #@param {type: "slider", min: 1, max: 1000}
Group_frames = 1  #@param {type: "slider", min: 1, max: 10}
#Binning_factor = 2 #@param {type: "slider", min: 1, max: 6}
#@markdown Gain reference correction options
Gain_reference_image = ''  #@param {type: "string"}
Gain_rotate_angle = 0 #@param {type: "slider", min: 0, max: 270,step:90}
Gain_flip = 'None' #@param ["None", "Flip upside down", "Flip left to right"] {allow-input: true}

import subprocess
import sys

if len(Gain_reference_image) >0:
   if Gain_reference_image.split('.')[-1] != 'mrc':
    print("Error: Gain reference file must be in .mrc format")
    sys.exit()
if '/content/' in Movie: 
    Movie=Movie
else:
    Movie='/content/'+Movie

print('Running RELION motion correction using the following parameters:\n')
print('Movie = %s' %(Movie)) 
print('Patch X= %i' %(Patch_X))
print('Patch Y= %i' %(Patch_Y))
print('B-Factor = %i' %(B_Factor))
print('Group frames = %i' %(Group_frames))

Binning_factor = 1

if len(Gain_reference_image)>0:
    Gain_rot_value=0
    if Gain_rotate_angle == 90:
        Gain_rot_value=1
    if Gain_rotate_angle == 180:
        Gain_rot_value=2
    if Gain_rotate_angle == 270:
        Gain_rot_value=3
    Gain_flip_value =0
    if Gain_flip == 'Flip upside down':
      Gain_flip_value =1
    if Gain_flip == 'Flip left to right':
      Gain_flip_value = 2
    if '/content' in Gain_reference_image: 
        Gain_reference_image=Gain_reference_image
    else: 
        Gain_reference_image='/content/'+Gain_reference_image
    gain_ref_cmd='--gainref "%s" --gain_flip %i' %(Gain_reference_image,Gain_flip_value)
    print('Gain reference image = %s' %(Gain_reference_image))
    print('Gain flip = %s (%i)' %(Gain_flip,Gain_flip_value))
    print('Gain rotate angle = %s (%i)' %(Gain_rotate_angle,Gain_rot_value))
if len(Gain_reference_image) == 0: 
    gain_ref_cmd=''
    print('No gain reference file provided')

cmd='''rm -rf relion-testing 
mkdir relion-testing 
cd relion-testing/
mkdir MotionCorr
mkdir movies/
cp "%s" movies/%s
ls movies/ > info.txt
/content/relion/build/bin/relion_run_motioncorr --i 'movies/*.tif' --o MotionCorr --patch_x %i --patch_y %i --group_frames %i --bfactor %i --bin_factor %i --use_own --angpix 0.66 --voltage 300 --j 2 %s ''' %(Movie,Movie.split('/')[-1],Patch_X,Patch_Y,Group_frames,B_Factor,Binning_factor,gain_ref_cmd)
#print(cmd)
subprocess.Popen(cmd,shell=True).wait()
print('...finished.\n')
#Import python library to read MRC files: mrcfile
import mrcfile

#Import python plotting software
import matplotlib.pyplot as plt
import matplotlib

#Specify micrograph path:
input_micrograph='/content/relion-testing/MotionCorr/movies/14sep05c_c_00003gr_00014sq_00006hl_00003es_frames_bin2.mrc'

#Read micrograph into python-readable array:
micrograph_array=mrcfile.open(input_micrograph).data
print('Reading micrograph into NumPy array...\n')

#Print micrograph info
print('Micrograph dimensions (X,Y): %s,%s' %(mrcfile.open(input_micrograph).data.shape[0],mrcfile.open(input_micrograph).data.shape[1]))
print('Micrograph pixel size (reading from MRC header): %0.3fA/pix' %(mrcfile.open(input_micrograph).voxel_size['x']))

#Display micrograph
matplotlib.rcParams['figure.figsize'] = [15, 15]
plt.imshow(micrograph_array,cmap='gray')